In [3]:
from fastapi import FastAPI
from pydantic import BaseModel
from fastapi.middleware.cors import CORSMiddleware
from detoxify import Detoxify
from contextlib import asynccontextmanager

# Detoxify modeli yalnızca bir kez yüklenir
print("🔄 Detoxify modeli yükleniyor...")
model = Detoxify('original')
print("✅ Detoxify modeli başarıyla yüklendi.")

# Lifespan handler tanımlayın
@asynccontextmanager
async def lifespan(app: FastAPI):
    print("✅ Backend uygulaması başlatılıyor...")
    yield  # Uygulama başlatılırken yapılacak işlemler burada biter
    print("🛑 Backend uygulaması kapatılıyor.")

# FastAPI uygulamasını başlat
app = FastAPI(lifespan=lifespan)

# CORS ayarları
app.add_middleware(
    CORSMiddleware,
    allow_origins=["http://localhost:3000"],  # React uygulamanızın URL'si
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Kullanıcıdan gelen veriyi tanımlayan sınıf
class InputText(BaseModel):
    text: str

# Ana sayfa
@app.get("/")
def root():
    print("🌐 Ana sayfa endpoint çağrıldı.")
    return {"message": "Detoxify API'ye hoş geldiniz!"}

# Cümle analiz endpoint'i
@app.post("/analyze")
def analyze_text(input_text: InputText):
    print(f"📨 Gelen metin: {input_text.text}")

    if not input_text.text.strip():
        print("⚠️ Boş metin gönderildi.")
        return {"error": "Lütfen analiz edilecek bir metin giriniz."}

    try:
        # Kullanıcının gönderdiği metni Detoxify modeli ile analiz et
        results = model.predict(input_text.text)
        print(f"🧪 Analiz sonuçları: {results}")
    except Exception as e:
        print(f"❌ Analiz sırasında bir hata oluştu: {str(e)}")
        return {"error": f"Analiz sırasında bir hata oluştu: {str(e)}"}

    # Türk Ceza Kanunu (TCK) eşleştirmesi
    tck_mapping = {
        "threat": {"tck": "TCK 106", "açıklama": "Tehdit"},
        "insult": {"tck": "TCK 125", "açıklama": "Hakaret"},
        "obscene": {"tck": "TCK 226", "açıklama": "Müstehcenlik"},
        "toxicity": {"tck": "TCK 216", "açıklama": "Halkı kin ve düşmanlığa tahrik"},
        "severe_toxicity": {"tck": "TCK 216", "açıklama": "Ciddi toksik içerik"},
        "identity_attack": {"tck": "TCK 216", "açıklama": "Kimlik saldırısı"},
    }

    # Skorları TCK maddeleriyle eşleştir
    response = []
    for key, value in results.items():
        if key in tck_mapping and value > 0.2:  # Eşik değerini %20 olarak ayarladık
            response.append({
                "tur": key,
                "tck": tck_mapping[key]["tck"],
                "aciklama": tck_mapping[key]["açıklama"],
                "skor": round(value, 2)
            })

    print(f"🔍 Döndürülen sonuç: {response}")
    return {"analyze_results": response}


🔄 Detoxify modeli yükleniyor...
✅ Detoxify modeli başarıyla yüklendi.


In [1]:
import requests

# FastAPI uygulamanızın çalıştığı URL
BASE_URL = "http://127.0.0.1:8000/analyze"

# Test verileri
test_texts = [
    {"text": "Sen aptalsın!"},  # Hakaret içeren bir cümle
    {"text": "Seni öldüreceğim."},  # Tehdit içeren bir cümle
    {"text": "Bu tamamen masum bir cümle."},  # Zararsız bir cümle
    {"text": ""},  # Boş metin
]

# Testi gerçekleştir ve sonuçları yazdır
for idx, test_data in enumerate(test_texts):
    print(f"\n🔍 Test #{idx + 1} için gönderilen metin: {test_data['text']}")
    response = requests.post(BASE_URL, json=test_data)

    if response.status_code == 200:
        print("✅ Başarılı yanıt alındı.")
        print(f"Sonuçlar: {response.json()}")
    else:
        print(f"❌ Hata! HTTP Durum Kodu: {response.status_code}")
        print(f"Yanıt: {response.text}")



🔍 Test #1 için gönderilen metin: Sen aptalsın!


ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=8000): Max retries exceeded with url: /analyze (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000027AEBD01DC0>: Failed to establish a new connection: [WinError 10061] Hedef makine etkin olarak reddettiğinden bağlantı kurulamadı'))